<a href="https://colab.research.google.com/github/Bilal-Waleed/PIAIC-Q2-P1/blob/main/PIAIC_Q2_Project_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 1: LangChain Hello World Project**

In this Project, we will create a simple LangChain Colab Notebook that uses the Google Gemini Flash 2.0 model to answer user questions.

The project Github repo is: https://github.com/panaversity/learn-agentic-ai/tree/main/02_generative_ai_for_beginners/PROJECTS/01_hello_gemini

# **Next Steps:**

**Experiment with Prompts:** Add more prompt templates to see how the model responds.

**Add Memory:** Use LangChain’s memory feature to make the interaction multi-turn.

**Integrate Tools:** Extend the chain to include tools like database queries or APIs.

**Explore Gemini Features:** Adjust temperature, max tokens, and other parameters to optimize responses.


In [ ]:
!pip install langchain
!pip install langchain-google-genai

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

gemini_api_key = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    max_retries=3,
    temperature=0.2,
    max_output_tokens=200,
    api_key=gemini_api_key
)

memory = ConversationBufferWindowMemory(k=5, memory_key="chat_history", return_messages=True)

informative_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=(
        "The following is a conversation between you and a user. "
        "You are a helpful assistant. Always provide a detailed and factual answer. "
        "If the question is new, answer it from your knowledge base without depending on the conversation history.\n\n"
        "Conversation History:\n{chat_history}\n\n"
        "User's Current Question:\n{question}"
    )
)

concise_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=(
        "The following is a conversation between you and a user. "
        "You are a helpful assistant. Provide a concise answer to the question. "
        "If the question is new, answer it from your knowledge base without depending on the conversation history.\n\n"
        "Conversation History:\n{chat_history}\n\n"
        "User's Current Question:\n{question}"
    )
)

chain_informative = LLMChain(llm=llm, prompt=informative_prompt, memory=memory)
chain_concise = LLMChain(llm=llm, prompt=concise_prompt, memory=memory)

print("Welcome! You can ask questions.")
print("Type 'exit' to end the conversation.\n")

while True:
    user_question = input("Your question: ")

    if user_question.lower() == "exit":
        print("\nGoodbye! Have a great day!")
        break

    detailed_response = chain_informative.run({"question": user_question})
    print(f"\nDetailed Response: {detailed_response}\n")

    concise_choice = input("Do you want a concise version of the answer? Type 'yes' for concise or press Enter to skip, or 'exit' to quit: ").lower()

    if concise_choice == "exit":
        print("\nGoodbye! Have a great day!")
        break

    if concise_choice == "yes":
        concise_response = chain_concise.run({"question": user_question})
        print(f"\nConcise Response: {concise_response}\n")
